In [1]:
import mlflow
import requests
import pandas as pd
import mlflow.pyfunc
from sklearn.preprocessing import StandardScaler

In [3]:
mlflow.set_tracking_uri("https://dagshub.com/empleo-inducido/project-ml.mlflow")

model_name = "testing-model"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

2024/05/30 07:13:27 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 1.22.0, required: numpy==1.26.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [10]:
train = pd.read_csv('/home/project/data/processed/train/train-dengue.csv')
test = pd.read_csv('/home/project/data/processed/test/test-dengue.csv')

scaler = StandardScaler()

X_train = train.drop('TIPO_PACIENTE', axis=1)
y_train = train['TIPO_PACIENTE']
X_train = pd.get_dummies(X_train, columns=['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], drop_first=True)
X_train = scaler.fit_transform(X_train)

In [13]:
pd.get_dummies(train.drop('TIPO_PACIENTE', axis=1).columns, columns=['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], drop_first=True)

,CIRROSIS_HEPATICA,DEFUNCION,DIABETES,DIA_SIGN_SINTOMAS,EDAD_ANOS,EMBARAZO,ENFERMEDAD_RENAL,ENFERMEDAD_ULC_PEPTICA,ENTIDAD_RES,ESTATUS_CASO,HEMORRAGICOS,HIPERTENSION,INMUNOSUPR,MES_SIGN_SINTOMAS,MUNICIPIO_RES,RESULTADO_PCR,SEMANA_SIGN_SINTOMAS,SEXO
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [44]:
import pickle

with open('./scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

scaler = pickle.load(open('./scaler.pkl', 'rb'))


In [43]:
# URL del servicio local
url = "http://localhost:8000/predict/"

test = pd.read_csv('/home/project/data/processed/test/test-dengue.csv')
df = test.drop('TIPO_PACIENTE', axis=1)
data = pd.get_dummies(df, columns=['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], drop_first=True).sample(5)
data = scaler.transform(data)

# Enviar una solicitud POST al servidor
response = requests.post(url, json={"data": data.tolist()})

# Imprimir la respuesta
print(response.json())



{'prediction': [1, 1, 1, 1, 1]}
